Load libraries

In [14]:
library(tensorflow)
library(EBImage)

Read in and plot the data

In [9]:
data = read.table('data', sep = ',')
data[,5] = as.numeric(as.factor(data[,5]))

data

V1,V2,V3,V4,V5
5.1,3.5,1.4,0.2,1
4.9,3.0,1.4,0.2,1
4.7,3.2,1.3,0.2,1
4.6,3.1,1.5,0.2,1
5.0,3.6,1.4,0.2,1
5.4,3.9,1.7,0.4,1
4.6,3.4,1.4,0.3,1
5.0,3.4,1.5,0.2,1
4.4,2.9,1.4,0.2,1
4.9,3.1,1.5,0.1,1


split training and testing set

In [11]:
split = sample(x =  c(1:nrow(data)), size = round(0.8*nrow(data)) )
train = data[split,]
test = data[-split,]      

Set parameters

In [38]:
clas = as.integer(3)#number of classes
out = 0.5 #dropout
ds = 0.999 #gradient descent
lr = 1e-5 #learningrate
length = as.integer(4)

Define placeholders

In [15]:
x <- tf$placeholder(tf$float32, shape(NULL, length), 'x')
#target values
labels <- tf$placeholder(tf$int64, shape(NULL), 'labels')
#dropout rate
keep_prob <- tf$placeholder(tf$float32, shape(),'keep_prob')
#learningrate
lrate <- tf$placeholder(tf$float32, shape(), 'lrate')

Define variables

In [16]:
w_fc1 <- tf$Variable( tf$truncated_normal(shape(length, 5L ), stddev=0.1), 'w_fc1')
b_fc1 <- tf$Variable( tf$truncated_normal(shape(5L), stddev=0.1), 'b_fc1')

w_fc2 <- tf$Variable( tf$truncated_normal(shape(5L, 5L), stddev=0.1), 'w_output')
b_fc2 <- tf$Variable( tf$truncated_normal(shape(5L), stddev=0.1), 'b_output')

w_fc3 <- tf$Variable( tf$truncated_normal(shape(5L, clas), stddev=0.1), 'w_output')
b_fc3 <- tf$Variable( tf$truncated_normal(shape(clas), stddev=0.1), 'b_output')

Define Network

In [24]:
h_fc1 <- tf$nn$relu(tf$matmul(x, w_fc1) + b_fc1)
h_fc1_drop <- tf$nn$dropout(h_fc1, keep_prob)
h_fc2 <- tf$nn$relu(tf$matmul(h_fc1_drop, w_fc2) + b_fc2)
h_fc2_drop <- tf$nn$dropout(h_fc2, keep_prob)
h_fc3 <- tf$nn$relu(tf$matmul(h_fc2_drop, w_fc3) + b_fc3)
h_output <- tf$nn$softmax(h_fc3)

Define cost

In [25]:
cost = tf$nn$sigmoid_cross_entropy_with_logits( logits = h_output, labels = tf$one_hot(labels, clas))

Define optimizer

In [26]:
train_step <- tf$train$AdamOptimizer(lrate)$minimize(cost)

Define some variables to print along the way

In [27]:
correct_prediction <- tf$equal(tf$argmax(h_output, 1L),labels)
accuracy <- tf$reduce_mean(tf$cast(correct_prediction, tf$float32))

Define the session

In [28]:
sess <- tf$InteractiveSession()
sess$run(tf$global_variables_initializer())

Train the network

In [40]:
for (i in 1:2000) {
  
  batch_input = as.matrix(train[,1:4])
batch_labels = as.vector(train[,5])
  #train met gradient descent
  sess$run(train_step, feed_dict = dict(x = batch_input , labels = batch_labels , keep_prob = out, lrate = ds^i*lr))
  
  
  
  
  
  #valideer om de 100 keer hoe het gaat op de testset
  if (i %% 100 == 0) {
    
    batch_input = as.matrix(test[,1:4])
batch_labels = as.vector(test[,5])
  #train met gradient descent
  
  #train met gradient descent
 test_accuracy =  sess$run(accuracy, feed_dict = dict(x = batch_input , labels = batch_labels , keep_prob = 1))
  
  
    print( paste("step:", i, "test accuracy:", test_accuracy) )    
  }
  
  

  
  
  

}
